# Question answering on the SQuAD dataset

## Colab requirements

Before restarting runtime (remeber to select GPU runtime)$\dots$

In [ ]:
!git clone https://github.com/Wadaboa/squad-question-answering.git
!pip install -r squad-question-answering/init/base_requirements.txt

After restarting runtime$\dots$

In [ ]:
import os, sys

sys.path.insert(0, "/content/squad-question-answering")
os.chdir("/content/squad-question-answering")

## Imports

In [1]:
import os
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import transformers
from transformers.trainer_utils import set_seed

import dataset
import model
import training
import tokenizer
import utils

%load_ext autoreload
%autoreload 2

## Initialization

### Matplotlib

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

### Weights & biases

In [3]:
%env WANDB_PROJECT=squad-qa
%env WANDB_ENTITY=wadaboa
%env WANDB_MODE=online
%env WANDB_RESUME=never
%env WANDB_WATCH=false
%env WANDB_SILENT=true

env: WANDB_PROJECT=squad-qa
env: WANDB_ENTITY=wadaboa
env: WANDB_MODE=online
env: WANDB_RESUME=never
env: WANDB_WATCH=false
env: WANDB_SILENT=true


In [4]:
!wandb login

In [5]:
!wandb enabled

W&B enabled.


### PyTorch and numpy

In [6]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

In [7]:
DEVICE = utils.get_device()
DEVICE

/root/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

## Preliminaries

### Raw data loading

In [8]:
DATA_FOLDER = os.path.join(os.getcwd(), "data")
TRAIN_DATA_FOLDER = os.path.join(DATA_FOLDER, "training")
TRAIN_SET_PATH = os.path.join(TRAIN_DATA_FOLDER, "training_set.json")
TEST_DATA_FOLDER = os.path.join(DATA_FOLDER, "testing")
TEST_SET_PATH = os.path.join(TEST_DATA_FOLDER, "test_set.json")

In [9]:
squad_dataset = dataset.SquadDataset(
    train_set_path=TRAIN_SET_PATH, test_set_path=TEST_SET_PATH, subset=0.00005
)

In [10]:
squad_dataset.raw_train_df

,answer_start,answer,title,context,question_id,question,context_id,answer_end
0,167,1735,Institute_of_technology,The world's first institution of technology or...,56de4d9ecffd8e1900b4b7e2,What year was the Banská Akadémia founded?,1860,171
1,793,SOS-based speed,Film_speed,The standard specifies how speed ratings shoul...,572674a05951b619008f7319,What is another speed that can also be reporte...,9354,808
2,421,Sumerian temples and palaces,Sumer,The most impressive and famous of Sumerian bui...,5730bb058ab72b1400f9c72c,Where were the use of advanced materials and t...,17505,449
3,192,mayor,"Ann_Arbor,_Michigan",Ann Arbor has a council-manager form of govern...,572781a5f1498d1400e8fa1f,Who is elected every even numbered year?,10585,197


In [11]:
squad_dataset.raw_test_df

,answer_start,answer,title,context,question_id,question,context_id,answer_end


### Embeddings

In [12]:
UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"

- FastText: 
    - _fasttext-wiki-news-subwords_ (dimensions: 300)
- GloVe:
    - _glove-twitter_ (dimensions: 25. 50, 100, 200)
    - _glove-wiki-gigaword_ (dimensions: 50, 100, 200, 300)
- Word2Vec:
    - _word2vec-google-news_ (dimensions: 300)
    - _word2vec-ruscorpora_ (dimensions: 300)

In [13]:
# See https://github.com/RaRe-Technologies/gensim-data
GLOVE_EMBEDDING_DIMENSION = 25
GLOVE_MODEL_NAME = "glove-twitter"
glove_embedding_model, glove_vocab = utils.load_embedding_model(
    GLOVE_MODEL_NAME,
    embedding_dimension=GLOVE_EMBEDDING_DIMENSION,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
)

In [14]:
glove_embedding_layer = model.get_embedding_module(
    glove_embedding_model, pad_id=glove_vocab[PAD_TOKEN]
)

## Utils

In [15]:
TRAINER_ARGS = utils.get_default_trainer_args()

### Standard data loading

In [39]:
MAX_CONTEXT_TOKENS = 300

In [40]:
standard_tokenizer = tokenizer.get_standard_tokenizer(
    glove_vocab,
    MAX_CONTEXT_TOKENS,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
    device=DEVICE,
)

In [42]:
standard_dm = dataset.SquadDataManager(squad_dataset, standard_tokenizer, device=DEVICE)

In [43]:
standard_dm.train_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56de4d9ecffd8e1900b4b7e2,What year was the Banská Akadémia founded?,Institute_of_technology,1860,The world's first institution of technology or...,[1735],[167],[171]
1,572781a5f1498d1400e8fa1f,Who is elected every even numbered year?,"Ann_Arbor,_Michigan",10585,Ann Arbor has a council-manager form of govern...,[mayor],[192],[197]
2,5730bb058ab72b1400f9c72c,Where were the use of advanced materials and t...,Sumer,17505,The most impressive and famous of Sumerian bui...,[Sumerian temples and palaces],[421],[449]


In [44]:
standard_dm.val_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,572674a05951b619008f7319,What is another speed that can also be reporte...,Film_speed,9354,The standard specifies how speed ratings shoul...,[SOS-based speed],[793],[808]


In [45]:
standard_dm.test_df

,index,answer,answer_start,answer_end


## Baseline model

In [51]:
%env WANDB_RUN_GROUP=baseline
baseline_run_name = utils.get_run_name()
baseline_args = partial(
    TRAINER_ARGS,
    output_dir=f"./checkpoints/{os.getenv('WANDB_RUN_GROUP')}/{baseline_run_name}",
    num_train_epochs=30,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
)

env: WANDB_RUN_GROUP=baseline


### Training and validation

In [53]:
baseline_model = model.QABaselineModel(
    glove_embedding_layer, MAX_CONTEXT_TOKENS, device=DEVICE
)
print(f"The baseline model has {baseline_model.count_parameters()} parameters")

The baseline model has 41000 parameters


In [54]:
baseline_optimizer = optim.Adam(baseline_model.parameters(), lr=1e-3)
baseline_lr_scheduler = transformers.get_constant_schedule(baseline_optimizer)

In [55]:
baseline_trainer = training.SquadTrainer(
    model=baseline_model,
    args=baseline_args(run_name=baseline_run_name),
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.train_dataset,
    eval_dataset=standard_dm.val_dataset,
    optimizers=(baseline_optimizer, baseline_lr_scheduler),
)

In [56]:
baseline_trainer.train()

RuntimeError: The size of tensor a (300) must match the size of tensor b (172) at non-singleton dimension 1

### Training only

In [62]:
baseline_model = model.QABaselineModel(
    glove_embedding_layer, MAX_CONTEXT_TOKENS, device=DEVICE
)
print(f"The baseline model has {baseline_model.count_parameters()} parameters")

The baseline model has 41000 parameters


In [ ]:
baseline_optimizer = optim.Adam(baseline_model.parameters(), lr=1e-3)
baseline_lr_scheduler = transformers.get_constant_schedule(baseline_optimizer)

In [63]:
baseline_trainer = training.SquadTrainer(
    model=baseline_model,
    args=baseline_args(run_name=f"{baseline_run_name}-whole", evaluation_strategy="no"),
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.whole_dataset,
    optimizers=(baseline_optimizer, baseline_lr_scheduler),
)

In [64]:
baseline_trainer.train()

Step,Training Loss
1,5.984300
5,6.331100
10,6.195200
15,6.259200
20,6.214700
25,6.151500
30,6.280500
35,6.277100


TrainOutput(global_step=35, training_loss=6.234272003173828, metrics={'train_runtime': 79.2375, 'train_samples_per_second': 0.442, 'total_flos': 0, 'epoch': 5.0})

### Testing

In [65]:
baseline_test_output = baseline_trainer.predict(standard_dm.test_dataset)
baseline_test_output.metrics

{'test_loss': 6.0533928871154785,
 'test_accuracy': 0.048774934275635486,
 'test_precision': 0.04997091627749208,
 'test_recall': 0.539648141845944,
 'test_f1': 0.08496900937695989,
 'test_em': 0.0,
 'test_runtime': 1.8771,
 'test_samples_per_second': 96.959}

In [66]:
baseline_answers_path = "results/answers/baseline.json"
utils.save_answers(baseline_answers_path, baseline_test_output.predictions[-1])
wandb.save(baseline_answers_path);
wandb.finish()

## BiDAF

In [46]:
%env WANDB_RUN_GROUP=bidaf
bidaf_run_name = utils.get_run_name()
bidaf_args = partial(
    TRAINER_ARGS,
    output_dir=f"./checkpoints/{os.getenv('WANDB_RUN_GROUP')}/{bidaf_run_name}",
    num_train_epochs=18,
    per_device_train_batch_size=60,
    per_device_eval_batch_size=60,
)

env: WANDB_RUN_GROUP=bidaf


### Training and validation

In [47]:
bidaf_model = model.QABiDAFModel(glove_embedding_layer, device=DEVICE)
print(f"The BiDAF model has {bidaf_model.count_parameters()} parameters")

The BiDAF model has 2052700 parameters


In [48]:
bidaf_optimizer = optim.Adadelta(bidaf_model.parameters(), lr=0.5)
bidaf_lr_scheduler = transformers.get_constant_schedule(bidaf_optimizer)

In [49]:
bidaf_trainer = training.SquadTrainer(
    model=bidaf_model,
    args=bidaf_args(run_name=bidaf_run_name),
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.train_dataset,
    eval_dataset=standard_dm.val_dataset,
    optimizers=(bidaf_optimizer, bidaf_lr_scheduler),
)

In [50]:
bidaf_trainer.train()

torch.Size([3, 172, 100])
torch.Size([3, 172, 200])
torch.Size([3, 172, 14]) torch.Size([3, 14])
torch.Size([3, 172, 200]) torch.Size([3, 172, 1]) torch.Size([3, 172])


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Em,Runtime,Samples Per Second
1,6.141500,7.783813,0.000000,0.000000,0.000000,0.000000,0.000000,0.281600,3.551000


torch.Size([1, 194, 100])
torch.Size([1, 194, 200])
torch.Size([1, 194, 13]) torch.Size([1, 13])
torch.Size([1, 194, 200]) torch.Size([1, 194, 1]) torch.Size([1, 194])
torch.Size([3, 172, 100])
torch.Size([3, 172, 200])
torch.Size([3, 172, 14]) torch.Size([3, 14])
torch.Size([3, 172, 200]) torch.Size([3, 172, 1]) torch.Size([3, 172])
torch.Size([1, 194, 100])
torch.Size([1, 194, 200])
torch.Size([1, 194, 13]) torch.Size([1, 13])
torch.Size([1, 194, 200]) torch.Size([1, 194, 1]) torch.Size([1, 194])


KeyboardInterrupt: 

### Training only

In [ ]:
bidaf_model = model.QABiDAFModel(glove_embedding_layer, device=DEVICE)
print(f"The BiDAF model has {bidaf_model.count_parameters()} parameters")

In [ ]:
bidaf_optimizer = optim.Adadelta(bidaf_model.parameters(), lr=0.5)
bidaf_lr_scheduler = transformers.get_constant_schedule(bidaf_optimizer)

In [ ]:
bidaf_trainer = training.SquadTrainer(
    model=bidaf_model,
    args=bidaf_args(run_name=f"{bidaf_run_name}-whole", evaluation_strategy="no"),
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.whole_dataset,
    optimizers=(bidaf_optimizer, bidaf_lr_scheduler),
)

In [ ]:
bidaf_trainer.train()

### Testing

In [45]:
bidaf_test_output = bidaf_trainer.predict(standard_dm.test_dataset)
bidaf_test_output.metrics

{'test_loss': 8.894977569580078,
 'test_f1': 0.0,
 'test_accuracy': 0.0,
 'test_em': 0.0,
 'test_runtime': 0.2739,
 'test_samples_per_second': 3.651}

In [46]:
bidaf_answers_path = "results/answers/bidaf.json"
utils.save_answers(bidaf_answers_path, bidaf_test_output.predictions[-1])
wandb.save(bidaf_answers_path);
wandb.finish()

## Transformers data loading

In [14]:
MAX_BERT_TOKENS = 512

In [15]:
bert_tokenizer = tokenizer.get_bert_tokenizer(max_tokens=MAX_BERT_TOKENS, device=DEVICE)

In [16]:
bert_dm = dataset.SquadDataManager(squad_dataset, bert_tokenizer, device=DEVICE)

In [17]:
bert_dm.train_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56cdd28562d2951400fa68bd,Who does M fight with?,Spectre_(2015_film),470,Bond and Swann return to London where they mee...,[C],[105],[106]
1,56de4d9ecffd8e1900b4b7e2,What year was the Banská Akadémia founded?,Institute_of_technology,1860,The world's first institution of technology or...,[1735],[167],[171]
2,572674a05951b619008f7319,What is another speed that can also be reporte...,Film_speed,9354,The standard specifies how speed ratings shoul...,[SOS-based speed],[793],[808]
3,5726ef98708984140094d66e,What conferences became a requirement after Va...,Pope_Paul_VI,10862,Some critiqued Paul VI's decision; the newly c...,[National Bishop Conferences],[347],[374]
4,572843ce4b864d190016485c,What was the purpose of top secret ICBM commit...,John_von_Neumann,11497,"Shortly before his death, when he was already ...",[decide on the feasibility of building an ICBM...,[194],[284]
5,5730bb058ab72b1400f9c72c,Where were the use of advanced materials and t...,Sumer,17505,The most impressive and famous of Sumerian bui...,[Sumerian temples and palaces],[421],[449]


In [18]:
bert_dm.val_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,570e1a2a0dc6ce1900204dbf,How many species of fungi have been found on A...,Antarctica,6902,About 1150 species of fungi have been recorded...,[1150],[6],[10]
1,572781a5f1498d1400e8fa1f,Who is elected every even numbered year?,"Ann_Arbor,_Michigan",10585,Ann Arbor has a council-manager form of govern...,[mayor],[192],[197]


In [19]:
bert_dm.test_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,57379ed81c456719005744d7,In what way do idea strings transmit tesion fo...,Force,2058,Tension forces can be modeled using ideal stri...,[instantaneously in action-reaction pairs],[250],[290]


## BERT

In [22]:
%env WANDB_RUN_GROUP=bert
bert_run_name = utils.get_run_name()
bert_args = partial(
    TRAINER_ARGS,
    output_dir=f"./checkpoints/{os.getenv('WANDB_RUN_GROUP')}/{bert_run_name}",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)

env: WANDB_RUN_GROUP=bert


### Training and validation

In [20]:
bert_model = model.QABertModel(device=DEVICE)
print(f"The BERT model has {bert_model.count_parameters()} parameters")

In [ ]:
bert_optimizer = optim.Adam(bert_model.parameters(), lr=5e-5)
bert_lr_scheduler = transformers.get_constant_schedule(bert_optimizer)

In [ ]:
bert_trainer = training.SquadTrainer(
    model=bert_model,
    args=bert_args(run_name=bert_run_name),
    data_collator=bert_dm.tokenizer,
    train_dataset=bert_dm.train_dataset,
    eval_dataset=bert_dm.val_dataset,
    optimizers=(bert_optimizer, bert_lr_scheduler),
)

In [23]:
bert_trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy,Em,Runtime,Samples Per Second
1,6.527200,6.653205,0.166667,0.250000,0.000000,1.042400,1.919000
2,6.527200,7.183872,0.166667,0.250000,0.000000,1.067100,1.874000
3,6.527200,7.202362,0.166667,0.250000,0.000000,0.952700,2.099000


TrainOutput(global_step=3, training_loss=5.523826281229655, metrics={'train_runtime': 45.0591, 'train_samples_per_second': 0.067, 'total_flos': 0, 'epoch': 3.0})

### Training only

In [ ]:
bert_model = model.QABertModel(device=DEVICE)
print(f"The BERT model has {bert_model.count_parameters()} parameters")

In [ ]:
bert_optimizer = optim.Adam(bert_model.parameters(), lr=5e-5)
bert_lr_scheduler = transformers.get_constant_schedule(bert_optimizer)

In [ ]:
bert_trainer = training.SquadTrainer(
    model=bert_model,
    args=bert_args(run_name=f"{bert_run_name}-whole", evaluation_strategy="no"),
    data_collator=bert_dm.tokenizer,
    train_dataset=bert_dm.whole_dataset,
    optimizers=(bert_optimizer, bert_lr_scheduler),
)

In [ ]:
bert_trainer.train()

### Testing

In [24]:
bert_test_output = bert_trainer.predict(bert_dm.test_dataset)
bert_test_output.metrics

{'test_loss': 9.000775337219238,
 'test_f1': 0.0,
 'test_accuracy': 0.0,
 'test_em': 0.0,
 'test_runtime': 0.6044,
 'test_samples_per_second': 1.655}

In [25]:
bert_answers_path = "results/answers/bert.json"
utils.save_answers(bert_answers_path, bert_test_output.predictions[-1])
wandb.save(bert_answers_path);
wandb.finish()